## Generate the results for Tab 4 [done]

In [1]:
import numpy as np
import pandas as pd
import glob



In [2]:
file_path = '/data/UKBB/SSL/reports/mtl/main/*csv'


In [3]:
file_list = glob.glob(file_path)

In [4]:
len(file_list)

28

In [5]:
file_list

['/data/UKBB/SSL/reports/mtl/main/rowlands_flip_net_ft_100k_freeze_-1_1.csv',
 '/data/UKBB/SSL/reports/mtl/main/oppo_flip_net_ft_100k_-1_1.csv',
 '/data/UKBB/SSL/reports/mtl/main/pamap_flip_net_ft_100k_freeze_-1_1.csv',
 '/data/UKBB/SSL/reports/mtl/main/rowlands_flip_net_ft_100k_-1_1.csv',
 '/data/UKBB/SSL/reports/mtl/main/oppo_flip_net_random_mlp_-1_1.csv',
 '/data/UKBB/SSL/reports/mtl/main/adl_flip_net_ft_100k_freeze_-1_1.csv',
 '/data/UKBB/SSL/reports/mtl/main/pamap_hand_crafted_-1_1.csv',
 '/data/UKBB/SSL/reports/mtl/main/oppo_flip_net_ft_100k_freeze_-1_1.csv',
 '/data/UKBB/SSL/reports/mtl/main/capture24_flip_net_ft_100k_freeze_-1_1.csv',
 '/data/UKBB/SSL/reports/mtl/main/wisdm_hand_crafted_-1_1.csv',
 '/data/UKBB/SSL/reports/mtl/main/oppo_hand_crafted_-1_1.csv',
 '/data/UKBB/SSL/reports/mtl/main/adl_hand_crafted_-1_1.csv',
 '/data/UKBB/SSL/reports/mtl/main/realworld_flip_net_ft_100k_-1_1.csv',
 '/data/UKBB/SSL/reports/mtl/main/adl_flip_net_random_mlp_-1_1.csv',
 '/data/UKBB/SSL/re

In [7]:
# get the ones with error bar
master_df = []
for file_path in file_list:
    file_name =  file_path.split('/')[-1]
    dataset = file_name.split('_')[0]
    model = '_'.join((file_name.split('_')[1:-1]))
    file_name = file_path.split('/')[-1]


    # epoch_count = file_name.split('_')[-1][:2]
    
    current_df = pd.read_csv(file_path)
    #print(len(current_df))


    current_df['id'] = current_df.index 
    current_df = pd.melt(current_df, id_vars=['id'])
    current_df.drop(['id'], inplace=True, axis=1)

    current_df['dataset'] = dataset
    # print(file_path, model, subject_num)
    
    current_df['model'] = model  


    if len(master_df) == 0:
        master_df = current_df
    else:
        master_df = pd.concat([master_df, current_df])
master_df = master_df.rename(columns={"variable": "metric"}) 

In [8]:
np.unique(master_df['dataset'])

array(['adl', 'capture24', 'oppo', 'pamap', 'realworld', 'rowlands',
       'wisdm'], dtype=object)

In [9]:
np.unique(master_df['model'])

array(['flip_net_ft_100k_-1', 'flip_net_ft_100k_freeze_-1',
       'flip_net_random_mlp_-1', 'hand_crafted_-1'], dtype=object)

In [10]:
master_df[master_df['dataset'] == 'adl']

,metric,value,dataset,model
0,precision,0.648430,adl,flip_net_ft_100k_freeze_-1
1,precision,0.457255,adl,flip_net_ft_100k_freeze_-1
2,precision,0.750000,adl,flip_net_ft_100k_freeze_-1
3,precision,0.585726,adl,flip_net_ft_100k_freeze_-1
4,precision,0.646825,adl,flip_net_ft_100k_freeze_-1
...,...,...,...,...
23,kappa,0.922246,adl,flip_net_ft_100k_-1
24,kappa,0.637681,adl,flip_net_ft_100k_-1
25,kappa,0.935238,adl,flip_net_ft_100k_-1
26,kappa,0.978468,adl,flip_net_ft_100k_-1


In [11]:
master_df = master_df[(master_df['metric'] == 'f1') |  (master_df['metric'] == 'kappa') ]

In [12]:
key=['dataset','model', 'metric']
ok = master_df.groupby(key).describe()

In [13]:
ok = ok.reset_index(['dataset', 'metric' , 'model' ])

In [14]:
with pd.option_context('display.float_format', '{:0.3f}'.format):
    print(ok)

      dataset                       model metric   value                    \
                                                   count  mean   std   min   
0         adl         flip_net_ft_100k_-1     f1   7.000 0.829 0.101 0.700   
1         adl         flip_net_ft_100k_-1  kappa   7.000 0.849 0.113 0.638   
2         adl  flip_net_ft_100k_freeze_-1     f1   7.000 0.645 0.107 0.482   
3         adl  flip_net_ft_100k_freeze_-1  kappa   7.000 0.654 0.123 0.440   
4         adl      flip_net_random_mlp_-1     f1   7.000 0.414 0.179 0.109   
5         adl      flip_net_random_mlp_-1  kappa   7.000 0.368 0.198 0.015   
6         adl             hand_crafted_-1     f1   7.000 0.764 0.180 0.538   
7         adl             hand_crafted_-1  kappa   7.000 0.720 0.199 0.452   
8   capture24         flip_net_ft_100k_-1     f1 155.000 0.726 0.093 0.420   
9   capture24         flip_net_ft_100k_-1  kappa 155.000 0.737 0.087 0.434   
10  capture24  flip_net_ft_100k_freeze_-1     f1 155.000 0.723 0

In [ ]:
def get_relative_boost(my_df, dataset):
    task_df = my_df[my_df.dataset == dataset]
    # Uncomment for full experimets 
    #data_set_order = ['capture24', 'rowlands', 'wisdm', 'realworld', 'oppo', 'pamap', 'adl']
    model_order = ['hand_crafted_-1', 'flip_net_random_mlp_-1', 'flip_net_ft_100k_freeze_-1',
                  'flip_net_ft_100k_-1']

    res_str_f1 = ""
    res_str_kappa = ""

    for my_model in model_order:
        f1_df = task_df[task_df.metric == 'f1']
        kappa_df = task_df[task_df.metric == 'kappa']
        
        f1_mean= f1_df[f1_df['model'] == my_model].iloc[0].value['mean']
        f1_std = f1_df[f1_df['model'] == my_model].iloc[0].value['std']
        kappa_mean= kappa_df[kappa_df['model'] == my_model].iloc[0].value['mean']
        kappa_std = kappa_df[kappa_df['model'] == my_model].iloc[0].value['std']
        
        if len(res_str_f1) == 0:
            res_str_f1 = r'%.3f $ \pm $ %.3f' % (f1_mean, f1_std)
            res_str_kappa = r'%.3f $ \pm $ %.3f' % (kappa_mean, kappa_std)
        else:
            res_str_f1 = res_str_f1 + r' & %.3f $ \pm $ %.3f' % (f1_mean, f1_std)
            res_str_kappa = res_str_kappa  + r' & %.3f $ \pm $ %.3f' % (kappa_mean, kappa_std)
    print(res_str_f1.replace('0.', '.'))
    print(res_str_kappa.replace('0.', '.'))
    return diff

In [23]:
def get_row_str(my_df, dataset):
    task_df = my_df[my_df.dataset == dataset]
    # Uncomment for full experimets 
    #data_set_order = ['capture24', 'rowlands', 'wisdm', 'realworld', 'oppo', 'pamap', 'adl']
    model_order = ['hand_crafted_-1', 'flip_net_random_mlp_-1', 'flip_net_ft_100k_freeze_-1',
                  'flip_net_ft_100k_-1']

    res_str_f1 = ""
    res_str_kappa = ""

    for my_model in model_order:
        f1_df = task_df[task_df.metric == 'f1']
        kappa_df = task_df[task_df.metric == 'kappa']
        
        f1_mean= f1_df[f1_df['model'] == my_model].iloc[0].value['mean']
        f1_std = f1_df[f1_df['model'] == my_model].iloc[0].value['std']
        kappa_mean= kappa_df[kappa_df['model'] == my_model].iloc[0].value['mean']
        kappa_std = kappa_df[kappa_df['model'] == my_model].iloc[0].value['std']
        
        if len(res_str_f1) == 0:
            res_str_f1 = r'%.3f $ \pm $ %.3f' % (f1_mean, f1_std)
            res_str_kappa = r'%.3f $ \pm $ %.3f' % (kappa_mean, kappa_std)
        else:
            res_str_f1 = res_str_f1 + r' & %.3f $ \pm $ %.3f' % (f1_mean, f1_std)
            res_str_kappa = res_str_kappa  + r' & %.3f $ \pm $ %.3f' % (kappa_mean, kappa_std)
    print(res_str_f1.replace('0.', '.'))
    print(res_str_kappa.replace('0.', '.'))
    return diff

In [16]:
get_row_str(ok, 'capture24')


.694 $ \pm $ .099 & .708 $ \pm $ .094 & .723 $ \pm $ .097 & .726 $ \pm $ .093
.683 $ \pm $ .101 & .703 $ \pm $ .092 & .718 $ \pm $ .090 & .737 $ \pm $ .087


In [17]:
get_row_str(ok, 'rowlands')


.700 $ \pm $ .090 & .696 $ \pm $ .106 & .724 $ \pm $ .081 & .796 $ \pm $ .093
.830 $ \pm $ .086 & .810 $ \pm $ .098 & .850 $ \pm $ .062 & .874 $ \pm $ .073


In [18]:
get_row_str(ok, 'wisdm')

.711 $ \pm $ .149 & .684 $ \pm $ .123 & .759 $ \pm $ .121 & .810 $ \pm $ .127
.715 $ \pm $ .153 & .685 $ \pm $ .124 & .758 $ \pm $ .121 & .809 $ \pm $ .126


In [19]:
get_row_str(ok, 'realworld')

.731 $ \pm $ .119 & .705 $ \pm $ .062 & .764 $ \pm $ .052 & .792 $ \pm $ .075
.680 $ \pm $ .142 & .638 $ \pm $ .079 & .703 $ \pm $ .063 & .739 $ \pm $ .086


In [20]:
get_row_str(ok, 'oppo')

.416 $ \pm $ .185 & .383 $ \pm $ .124 & .570 $ \pm $ .078 & .595 $ \pm $ .085
.318 $ \pm $ .206 & .238 $ \pm $ .154 & .435 $ \pm $ .092 & .471 $ \pm $ .104


In [21]:
get_row_str(ok, 'pamap')

.753 $ \pm $ .093 & .605 $ \pm $ .086 & .725 $ \pm $ .054 & .789 $ \pm $ .054
.744 $ \pm $ .101 & .596 $ \pm $ .086 & .717 $ \pm $ .057 & .769 $ \pm $ .059


In [22]:
get_row_str(ok, 'adl')

.764 $ \pm $ .180 & .414 $ \pm $ .179 & .645 $ \pm $ .107 & .829 $ \pm $ .101
.720 $ \pm $ .199 & .368 $ \pm $ .198 & .654 $ \pm $ .123 & .849 $ \pm $ .113
